## Ejercicios de pair programming 10 enero : ETL 2.1

In [189]:
import requests
import pandas as pd
import numpy as np

pd.options.display.max_columns = None

El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:

- Cargaremos los dos ficheros de datos
- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:
    - USA
    - Australia
    - New Zealand
    - South Africa
    - Papua New Guinea


In [190]:
#Abrimos el archivo y lo guardamos en tiburon
tiburon = pd.read_csv("../datos/tiburon8.csv", index_col=0)

In [191]:
tiburon.head(2)

,case_number,year,type,country,activity,injury,mes,deceso,genero,especies,edad
0,2018.06.25,2018,Boating,usa,Paddling,"No injury to occupant, outrigger canoe and pad...",Jun,N,F,White,57.0
1,2018.06.03.a,2018,Unprovoked,brazil,Swimming,FATAL,Jun,Y,M,Tiger,18.0


In [192]:
# filtramos df de tiburon por los países que necesitamos
df = tiburon[tiburon['country'].isin(['usa', 'australia', 'new zealand', 'south africa', 'papua new guinea'])]

In [193]:
df.country.unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

Del dataframe de los datos climáticos seleccionaremos todas las columnas.

Cuando ya tengamos todos los datos deseados juntaremos los dos csv

Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.

Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. 

A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. 

Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [194]:
#Abrimos el archivo y lo guardamos en clima
clima = pd.read_csv("../datos/meteo_tiburon.csv", index_col=0)

In [195]:
clima.head(3)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 210, 'speed':...",12,15,4,1026,none,0,0,270,2,USA
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 345, 'speed':...",12,15,3,1027,none,0,0,15,2,USA
2,9,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 180, 'speed':...",12,15,2,1028,none,0,0,310,2,USA


In [196]:
# importamos ast para poder castear las columnas y poder tener listas en vez de string en las columnas para luego poder aplicar el pd.Series y aplicamos en las columnas que queremos limpiar
import ast
clima['rh_profile']= clima['rh_profile'].apply(ast.literal_eval)
clima['wind_profile']= clima['wind_profile'].apply(ast.literal_eval)

In [197]:
clima.head(3)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 210, 'speed':...",12,15,4,1026,none,0,0,270,2,USA
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 345, 'speed':...",12,15,3,1027,none,0,0,15,2,USA
2,9,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 180, 'speed':...",12,15,2,1028,none,0,0,310,2,USA


In [198]:
#Aplicamos pd.Series para separar cada diccionario de la columna rh_profile de la lista por columnas y lo guardamos en rh
rh = clima['rh_profile'].apply(pd.Series)
rh.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': 3}","{'layer': '900mb', 'rh': 0}","{'layer': '850mb', 'rh': -2}","{'layer': '800mb', 'rh': -1}","{'layer': '750mb', 'rh': 0}","{'layer': '700mb', 'rh': 2}","{'layer': '650mb', 'rh': 4}","{'layer': '600mb', 'rh': 6}","{'layer': '550mb', 'rh': 7}","{'layer': '500mb', 'rh': 7}","{'layer': '450mb', 'rh': 5}","{'layer': '400mb', 'rh': 1}","{'layer': '350mb', 'rh': -1}","{'layer': '300mb', 'rh': 2}","{'layer': '250mb', 'rh': 2}","{'layer': '200mb', 'rh': 4}"
1,"{'layer': '950mb', 'rh': 2}","{'layer': '900mb', 'rh': 0}","{'layer': '850mb', 'rh': 0}","{'layer': '800mb', 'rh': 1}","{'layer': '750mb', 'rh': 3}","{'layer': '700mb', 'rh': 5}","{'layer': '650mb', 'rh': 7}","{'layer': '600mb', 'rh': 7}","{'layer': '550mb', 'rh': 5}","{'layer': '500mb', 'rh': 3}","{'layer': '450mb', 'rh': 2}","{'layer': '400mb', 'rh': 0}","{'layer': '350mb', 'rh': 4}","{'layer': '300mb', 'rh': 5}","{'layer': '250mb', 'rh': 7}","{'layer': '200mb', 'rh': 7}"
2,"{'layer': '950mb', 'rh': 2}","{'layer': '900mb', 'rh': 1}","{'layer': '850mb', 'rh': 4}","{'layer': '800mb', 'rh': 6}","{'layer': '750mb', 'rh': 6}","{'layer': '700mb', 'rh': 6}","{'layer': '650mb', 'rh': 7}","{'layer': '600mb', 'rh': 7}","{'layer': '550mb', 'rh': 6}","{'layer': '500mb', 'rh': 4}","{'layer': '450mb', 'rh': 1}","{'layer': '400mb', 'rh': -1}","{'layer': '350mb', 'rh': 1}","{'layer': '300mb', 'rh': 3}","{'layer': '250mb', 'rh': 6}","{'layer': '200mb', 'rh': 5}"
3,"{'layer': '950mb', 'rh': 1}","{'layer': '900mb', 'rh': 2}","{'layer': '850mb', 'rh': 5}","{'layer': '800mb', 'rh': 5}","{'layer': '750mb', 'rh': 3}","{'layer': '700mb', 'rh': 3}","{'layer': '650mb', 'rh': 3}","{'layer': '600mb', 'rh': 5}","{'layer': '550mb', 'rh': 6}","{'layer': '500mb', 'rh': 6}","{'layer': '450mb', 'rh': 2}","{'layer': '400mb', 'rh': -1}","{'layer': '350mb', 'rh': 0}","{'layer': '300mb', 'rh': 11}","{'layer': '250mb', 'rh': 1}","{'layer': '200mb', 'rh': 5}"
4,"{'layer': '950mb', 'rh': 1}","{'layer': '900mb', 'rh': 4}","{'layer': '850mb', 'rh': 5}","{'layer': '800mb', 'rh': 5}","{'layer': '750mb', 'rh': 3}","{'layer': '700mb', 'rh': 2}","{'layer': '650mb', 'rh': 2}","{'layer': '600mb', 'rh': 2}","{'layer': '550mb', 'rh': 3}","{'layer': '500mb', 'rh': 4}","{'layer': '450mb', 'rh': 6}","{'layer': '400mb', 'rh': 5}","{'layer': '350mb', 'rh': 4}","{'layer': '300mb', 'rh': 0}","{'layer': '250mb', 'rh': 1}","{'layer': '200mb', 'rh': 5}"


In [199]:
#Probamos para sacar lo que será el nombre de la columna
rh.iloc[0][0]['layer']

'950mb'

In [200]:
#Probamos para sacar lo que será los valores del df
print(list(rh[0].apply(pd.Series)["rh"]))


[3, 2, 2, 1, 1, 4, 5, 6, 6, 5, 5, 4, 6, 6, 4, 5, 4, 7, 8, 3, 2, 4, 7, 6, 3, 1, 1, 2, 2, 6, 4, 3, 3, 2, 2, 1, 2, 2, 3, 4, 4, 4, 4, 3, 3, 2, 3, 3, 4, 7, 6, 6, 7, 8, 10, 8, 9, 9, 5, 2, 2, 3, 4, 5, 13, 14, 15, 14, 12, 14, 14, 14, 14, 14, 14, 14, 14, 14, 13, 13, 14, 15, 14, 15, 14, 14, 15, 13, 14, 14, 14, 14, 13, 14, 13, 12, 14, 14, 14, 14, 14, 12, 13, 13, 13, 14, 15, 15, 15, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 13, 10, 11, 12, 11, 13, 11, 13, 13, 12, 12, 13, 13, 14, 15, 14, 14, 11, 10, 10, 10, 10, 12, 13, 13, 13, 13, 13, 13, 13, 14, 12, 12, 13, 13, 13, 13, 14, 13, 14, 13, 12, 13, 13, 12, 13, 13, 13, 12, 12, 12, 12, 12, 11, 10, 9, 10, 10, 10, 11, 13, 12, 13, 13, 13, 13, 12, 12, 13, 12, 12, 14, 13, 12, 15, 14, 12, 14, 15, 16, 16, 15, 15, 16, 14, 15, 13, 13, 13, 16, 15, 15, 15, 15, 14, 14, 14, 13, 12, 12, 12, 12, 12, 13, 14, 14, 14, 14, 13, 14, 13, 13, 11, 13, 13, 14, 14, 13, 13, 13, 13, 13, 13, 12, 11, 11, 10, 11, 11, 12, 13, 13, 13, 14, 13, 14, 12, 14, 14, 15, 14, 14, 14, 13, 14,

In [201]:
# Iteramos por las columnas de rh y definimos el nombre de la columna como rh_+valor de layer y los valores cada value de rh

for i in range(len(rh.columns)): 

    nombre = "rh_" + str(rh.iloc[0][i]['layer']) 

    valores = list(rh[i].apply(pd.Series)["rh"] )

    clima.insert(i, nombre, valores) # Insertamos cada columna al df de clima

In [202]:
clima.head(2)

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,0,-2,-1,0,2,4,6,7,7,5,1,-1,2,2,4,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 210, 'speed':...",12,15,4,1026,none,0,0,270,2,USA
1,2,0,0,1,3,5,7,7,5,3,2,0,4,5,7,7,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 345, 'speed':...",12,15,3,1027,none,0,0,15,2,USA


In [203]:
#Aplicamos pd.Series para separar cada diccionario de la columna wind_profile de la lista por columnas y lo guardamos en wind
wind = clima['wind_profile'].apply(pd.Series)
wind.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': 210, 'speed': 2}","{'layer': '900mb', 'direction': 250, 'speed': 2}","{'layer': '850mb', 'direction': 300, 'speed': 3}","{'layer': '800mb', 'direction': 305, 'speed': 4}","{'layer': '750mb', 'direction': 315, 'speed': 4}","{'layer': '700mb', 'direction': 320, 'speed': 5}","{'layer': '650mb', 'direction': 315, 'speed': 5}","{'layer': '600mb', 'direction': 310, 'speed': 5}","{'layer': '550mb', 'direction': 305, 'speed': 5}","{'layer': '500mb', 'direction': 305, 'speed': 6}","{'layer': '450mb', 'direction': 310, 'speed': 6}","{'layer': '400mb', 'direction': 325, 'speed': 6}","{'layer': '350mb', 'direction': 330, 'speed': 7}","{'layer': '300mb', 'direction': 335, 'speed': 7}","{'layer': '250mb', 'direction': 335, 'speed': 8}","{'layer': '200mb', 'direction': 340, 'speed': 8}"
1,"{'layer': '950mb', 'direction': 345, 'speed': 2}","{'layer': '900mb', 'direction': 35, 'speed': 2}","{'layer': '850mb', 'direction': 355, 'speed': 3}","{'layer': '800mb', 'direction': 325, 'speed': 3}","{'layer': '750mb', 'direction': 315, 'speed': 3}","{'layer': '700mb', 'direction': 310, 'speed': 4}","{'layer': '650mb', 'direction': 310, 'speed': 5}","{'layer': '600mb', 'direction': 305, 'speed': 5}","{'layer': '550mb', 'direction': 305, 'speed': 5}","{'layer': '500mb', 'direction': 310, 'speed': 5}","{'layer': '450mb', 'direction': 325, 'speed': 6}","{'layer': '400mb', 'direction': 330, 'speed': 7}","{'layer': '350mb', 'direction': 325, 'speed': 7}","{'layer': '300mb', 'direction': 325, 'speed': 7}","{'layer': '250mb', 'direction': 335, 'speed': 7}","{'layer': '200mb', 'direction': 340, 'speed': 8}"
2,"{'layer': '950mb', 'direction': 180, 'speed': 2}","{'layer': '900mb', 'direction': 0, 'speed': 2}","{'layer': '850mb', 'direction': 330, 'speed': 3}","{'layer': '800mb', 'direction': 335, 'speed': 4}","{'layer': '750mb', 'direction': 330, 'speed': 4}","{'layer': '700mb', 'direction': 330, 'speed': 5}","{'layer': '650mb', 'direction': 335, 'speed': 5}","{'layer': '600mb', 'direction': 330, 'speed': 5}","{'layer': '550mb', 'direction': 320, 'speed': 5}","{'layer': '500mb', 'direction': 325, 'speed': 6}","{'layer': '450mb', 'direction': 325, 'speed': 6}","{'layer': '400mb', 'direction': 320, 'speed': 6}","{'layer': '350mb', 'direction': 320, 'speed': 6}","{'layer': '300mb', 'direction': 335, 'speed': 6}","{'layer': '250mb', 'direction': 330, 'speed': 6}","{'layer': '200mb', 'direction': 345, 'speed': 7}"


In [204]:
# Como en el caso anterior iteramos por las columnas de wind y definimos dos nombres de columnas, una por dirección y otra por velocidad y los valores de cada key del diccionario

for i in range(len(wind.columns)): 

    nombre_dir = "wdir_" + str(wind.iloc[0][i]['layer']) 
    valores_dir = list(wind[i].apply(pd.Series)["direction"] )

    nombre_speed = "wspeed_" + str(wind.iloc[0][i]['layer']) 
    valores_speed = list(wind[i].apply(pd.Series)["speed"] )

    clima.insert(i, nombre_dir, valores_dir) # Insertamos las columnas generadas en el df de clima
    clima.insert(i, nombre_speed, valores_speed)

In [205]:
clima.head(2)

,wspeed_950mb,wspeed_900mb,wspeed_850mb,wspeed_800mb,wspeed_750mb,wspeed_700mb,wspeed_650mb,wspeed_600mb,wspeed_550mb,wspeed_500mb,wspeed_450mb,wspeed_400mb,wspeed_350mb,wspeed_300mb,wspeed_250mb,wspeed_200mb,wdir_200mb,wdir_250mb,wdir_300mb,wdir_350mb,wdir_400mb,wdir_450mb,wdir_500mb,wdir_550mb,wdir_600mb,wdir_650mb,wdir_700mb,wdir_750mb,wdir_800mb,wdir_850mb,wdir_900mb,wdir_950mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,2,2,3,4,4,5,5,5,5,6,6,6,7,7,8,8,340,335,335,330,325,310,305,305,310,315,320,315,305,300,250,210,3,0,-2,-1,0,2,4,6,7,7,5,1,-1,2,2,4,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 210, 'speed':...",12,15,4,1026,none,0,0,270,2,USA
1,2,2,3,3,3,4,5,5,5,5,6,7,7,7,7,8,340,335,325,325,330,325,310,305,305,310,310,315,325,355,35,345,2,0,0,1,3,5,7,7,5,3,2,0,4,5,7,7,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 345, 'speed':...",12,15,3,1027,none,0,0,15,2,USA


In [206]:
clima.shape

(320, 65)

In [207]:
clima = clima.groupby('pais').mean() #Agrupamos clima por cada país calculando la media de las columnas

/tmp/ipykernel_1699/269306169.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  clima = clima.groupby('pais').mean() #Agrupamos clima por cada país calculando la media de las columnas


In [208]:
clima.head()

,wspeed_950mb,wspeed_900mb,wspeed_850mb,wspeed_800mb,wspeed_750mb,wspeed_700mb,wspeed_650mb,wspeed_600mb,wspeed_550mb,wspeed_500mb,wspeed_450mb,wspeed_400mb,wspeed_350mb,wspeed_300mb,wspeed_250mb,wspeed_200mb,wdir_200mb,wdir_250mb,wdir_300mb,wdir_350mb,wdir_400mb,wdir_450mb,wdir_500mb,wdir_550mb,wdir_600mb,wdir_650mb,wdir_700mb,wdir_750mb,wdir_800mb,wdir_850mb,wdir_900mb,wdir_950mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
pais,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Australia,3.671875,3.640625,3.500000,3.406250,3.375000,3.390625,3.312500,3.421875,3.703125,3.843750,4.000000,4.031250,4.250000,4.593750,5.093750,5.671875,161.015625,151.875000,138.203125,123.203125,120.546875,121.015625,129.453125,133.906250,132.421875,136.484375,133.828125,136.250000,135.546875,129.609375,115.703125,121.406250,13.609375,11.234375,8.406250,6.796875,5.671875,4.187500,3.515625,2.796875,2.593750,2.375000,2.171875,2.078125,1.546875,1.640625,4.125000,6.984375,97.5,3.703125,-9999.0,-9999.0,-9999.0,25.890625,-3.750000,10.906250,1014.906250,2.859375,0.0,122.968750,3.296875
New Zealand,3.687500,3.625000,3.500000,3.500000,3.484375,3.546875,3.609375,3.765625,3.984375,4.140625,4.484375,4.796875,5.093750,5.406250,6.406250,7.500000,288.593750,276.484375,264.609375,254.843750,241.796875,233.515625,219.843750,216.328125,223.125000,218.984375,212.500000,194.765625,191.875000,178.359375,159.921875,157.890625,13.359375,12.906250,7.921875,5.296875,3.265625,2.671875,1.875000,1.703125,3.140625,4.640625,4.906250,5.109375,5.031250,4.046875,1.421875,-1.812500,97.5,6.734375,-9999.0,-9999.0,-9999.0,15.062500,9.093750,10.859375,1015.500000,3.203125,0.0,149.453125,3.359375
Papua New Guinea,3.828125,4.250000,4.296875,4.109375,3.406250,2.828125,2.437500,2.734375,3.218750,3.703125,4.062500,4.218750,4.265625,4.140625,4.687500,5.078125,221.562500,190.468750,220.859375,239.140625,243.281250,256.796875,260.859375,240.781250,196.953125,145.234375,79.843750,77.343750,80.390625,82.109375,82.031250,82.812500,13.578125,11.531250,9.343750,5.812500,2.531250,0.343750,1.296875,0.031250,-1.812500,-1.906250,0.296875,1.750000,2.218750,2.218750,2.125000,1.171875,97.5,3.546875,-9999.0,-9999.0,-9999.0,25.890625,-0.671875,11.500000,1010.265625,2.109375,0.0,84.375000,3.156250
South Africa,2.359375,2.265625,2.218750,2.140625,2.156250,2.234375,2.484375,2.781250,3.062500,3.375000,3.843750,4.140625,4.578125,5.093750,5.359375,5.968750,208.437500,209.687500,212.812500,216.953125,219.218750,216.875000,215.000000,211.406250,200.468750,203.125000,202.968750,196.250000,181.484375,159.062500,142.812500,147.656250,12.328125,8.703125,6.546875,7.421875,8.812500,9.953125,9.984375,8.296875,5.625000,4.562500,4.687500,4.953125,5.656250,6.312500,7.593750,9.390625,97.5,7.031250,-9999.0,-9999.0,-9999.0,23.421875,2.562500,10.156250,1019.484375,1.781250,0.0,152.500000,2.296875
USA,2.343750,2.515625,2.953125,3.296875,3.531250,3.937500,4.296875,4.484375,4.609375,4.734375,4.968750,5.046875,5.078125,5.421875,5.640625,5.484375,248.828125,222.500000,235.390625,240.468750,260.234375,271.640625,263.671875,267.656250,261.015625,261.562500,267.187500,271.718750,279.921875,276.875000,230.468750,222.812500,4.187500,4.000000,3.890625,3.640625,3.546875,3.265625,2.453125,1.703125,2.046875,3.281250,4.921875,5.046875,6.078125,6.656250,5.937500,5.734375,97.5,3.562500,-9999.0,-9999.0,-9999.0,12.718750,13.265625,4.812500,1022.968750,0.000000,0.0,203.750000,2.093750


In [209]:
clima = clima.reset_index() # resetamos en index para poder cambiar los nombres de los países a minusculas que es como los tenemos en df

In [210]:
#Definimos la función para transformar los nombres de los países a minusculas
def minusculas(col):
    col = col.lower()
    return col

In [211]:
clima['country'] = clima['pais'].apply(minusculas) #Aplicamos la función y generamos una nueva columna llamada country como en el df original

In [212]:
clima.drop(['pais'], axis= 1, inplace=True)  # Eliminamos la columna pais

In [213]:
clima.head()

,wspeed_950mb,wspeed_900mb,wspeed_850mb,wspeed_800mb,wspeed_750mb,wspeed_700mb,wspeed_650mb,wspeed_600mb,wspeed_550mb,wspeed_500mb,wspeed_450mb,wspeed_400mb,wspeed_350mb,wspeed_300mb,wspeed_250mb,wspeed_200mb,wdir_200mb,wdir_250mb,wdir_300mb,wdir_350mb,wdir_400mb,wdir_450mb,wdir_500mb,wdir_550mb,wdir_600mb,wdir_650mb,wdir_700mb,wdir_750mb,wdir_800mb,wdir_850mb,wdir_900mb,wdir_950mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3.671875,3.640625,3.500000,3.406250,3.375000,3.390625,3.312500,3.421875,3.703125,3.843750,4.000000,4.031250,4.250000,4.593750,5.093750,5.671875,161.015625,151.875000,138.203125,123.203125,120.546875,121.015625,129.453125,133.906250,132.421875,136.484375,133.828125,136.250000,135.546875,129.609375,115.703125,121.406250,13.609375,11.234375,8.406250,6.796875,5.671875,4.187500,3.515625,2.796875,2.593750,2.375000,2.171875,2.078125,1.546875,1.640625,4.125000,6.984375,97.5,3.703125,-9999.0,-9999.0,-9999.0,25.890625,-3.750000,10.906250,1014.906250,2.859375,0.0,122.968750,3.296875,australia
1,3.687500,3.625000,3.500000,3.500000,3.484375,3.546875,3.609375,3.765625,3.984375,4.140625,4.484375,4.796875,5.093750,5.406250,6.406250,7.500000,288.593750,276.484375,264.609375,254.843750,241.796875,233.515625,219.843750,216.328125,223.125000,218.984375,212.500000,194.765625,191.875000,178.359375,159.921875,157.890625,13.359375,12.906250,7.921875,5.296875,3.265625,2.671875,1.875000,1.703125,3.140625,4.640625,4.906250,5.109375,5.031250,4.046875,1.421875,-1.812500,97.5,6.734375,-9999.0,-9999.0,-9999.0,15.062500,9.093750,10.859375,1015.500000,3.203125,0.0,149.453125,3.359375,new zealand
2,3.828125,4.250000,4.296875,4.109375,3.406250,2.828125,2.437500,2.734375,3.218750,3.703125,4.062500,4.218750,4.265625,4.140625,4.687500,5.078125,221.562500,190.468750,220.859375,239.140625,243.281250,256.796875,260.859375,240.781250,196.953125,145.234375,79.843750,77.343750,80.390625,82.109375,82.031250,82.812500,13.578125,11.531250,9.343750,5.812500,2.531250,0.343750,1.296875,0.031250,-1.812500,-1.906250,0.296875,1.750000,2.218750,2.218750,2.125000,1.171875,97.5,3.546875,-9999.0,-9999.0,-9999.0,25.890625,-0.671875,11.500000,1010.265625,2.109375,0.0,84.375000,3.156250,papua new guinea
3,2.359375,2.265625,2.218750,2.140625,2.156250,2.234375,2.484375,2.781250,3.062500,3.375000,3.843750,4.140625,4.578125,5.093750,5.359375,5.968750,208.437500,209.687500,212.812500,216.953125,219.218750,216.875000,215.000000,211.406250,200.468750,203.125000,202.968750,196.250000,181.484375,159.062500,142.812500,147.656250,12.328125,8.703125,6.546875,7.421875,8.812500,9.953125,9.984375,8.296875,5.625000,4.562500,4.687500,4.953125,5.656250,6.312500,7.593750,9.390625,97.5,7.031250,-9999.0,-9999.0,-9999.0,23.421875,2.562500,10.156250,1019.484375,1.781250,0.0,152.500000,2.296875,south africa
4,2.343750,2.515625,2.953125,3.296875,3.531250,3.937500,4.296875,4.484375,4.609375,4.734375,4.968750,5.046875,5.078125,5.421875,5.640625,5.484375,248.828125,222.500000,235.390625,240.468750,260.234375,271.640625,263.671875,267.656250,261.015625,261.562500,267.187500,271.718750,279.921875,276.875000,230.468750,222.812500,4.187500,4.000000,3.890625,3.640625,3.546875,3.265625,2.453125,1.703125,2.046875,3.281250,4.921875,5.046875,6.078125,6.656250,5.937500,5.734375,97.5,3.562500,-9999.0,-9999.0,-9999.0,12.718750,13.265625,4.812500,1022.968750,0.000000,0.0,203.750000,2.093750,usa


In [214]:
df = df.merge(clima, how='left',  on='country') #Unimos clima a df con merge donde la columna country es igual

In [215]:
df.head(3)

,case_number,year,type,country,activity,injury,mes,deceso,genero,especies,edad,wspeed_950mb,wspeed_900mb,wspeed_850mb,wspeed_800mb,wspeed_750mb,wspeed_700mb,wspeed_650mb,wspeed_600mb,wspeed_550mb,wspeed_500mb,wspeed_450mb,wspeed_400mb,wspeed_350mb,wspeed_300mb,wspeed_250mb,wspeed_200mb,wdir_200mb,wdir_250mb,wdir_300mb,wdir_350mb,wdir_400mb,wdir_450mb,wdir_500mb,wdir_550mb,wdir_600mb,wdir_650mb,wdir_700mb,wdir_750mb,wdir_800mb,wdir_850mb,wdir_900mb,wdir_950mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,2018.06.25,2018,Boating,usa,Paddling,"No injury to occupant, outrigger canoe and pad...",Jun,N,F,White,57.0,2.343750,2.515625,2.953125,3.296875,3.53125,3.937500,4.296875,4.484375,4.609375,4.734375,4.96875,5.046875,5.078125,5.421875,5.640625,5.484375,248.828125,222.500,235.390625,240.468750,260.234375,271.640625,263.671875,267.65625,261.015625,261.562500,267.187500,271.71875,279.921875,276.875000,230.468750,222.81250,4.187500,4.000000,3.890625,3.640625,3.546875,3.265625,2.453125,1.703125,2.046875,3.28125,4.921875,5.046875,6.078125,6.656250,5.9375,5.734375,97.5,3.562500,-9999.0,-9999.0,-9999.0,12.718750,13.265625,4.81250,1022.96875,0.000000,0.0,203.75000,2.093750
1,2018.05.26.b,2018,Unprovoked,usa,Walking,Lower left leg bitten,May,N,M,Bull,15.0,2.343750,2.515625,2.953125,3.296875,3.53125,3.937500,4.296875,4.484375,4.609375,4.734375,4.96875,5.046875,5.078125,5.421875,5.640625,5.484375,248.828125,222.500,235.390625,240.468750,260.234375,271.640625,263.671875,267.65625,261.015625,261.562500,267.187500,271.71875,279.921875,276.875000,230.468750,222.81250,4.187500,4.000000,3.890625,3.640625,3.546875,3.265625,2.453125,1.703125,2.046875,3.28125,4.921875,5.046875,6.078125,6.656250,5.9375,5.734375,97.5,3.562500,-9999.0,-9999.0,-9999.0,12.718750,13.265625,4.81250,1022.96875,0.000000,0.0,203.75000,2.093750
2,2018.05.24,2018,Provoked,australia,Feeding sharks,Minor bite to hand by captive shark. PROVOKED ...,May,N,M,Grey,32.0,3.671875,3.640625,3.500000,3.406250,3.37500,3.390625,3.312500,3.421875,3.703125,3.843750,4.00000,4.031250,4.250000,4.593750,5.093750,5.671875,161.015625,151.875,138.203125,123.203125,120.546875,121.015625,129.453125,133.90625,132.421875,136.484375,133.828125,136.25000,135.546875,129.609375,115.703125,121.40625,13.609375,11.234375,8.406250,6.796875,5.671875,4.187500,3.515625,2.796875,2.593750,2.37500,2.171875,2.078125,1.546875,1.640625,4.1250,6.984375,97.5,3.703125,-9999.0,-9999.0,-9999.0,25.890625,-3.750000,10.90625,1014.90625,2.859375,0.0,122.96875,3.296875


In [216]:
df.to_csv('../datos/tiburon9.csv') #Guardamos nuestro df en tiburon9